<a href="https://colab.research.google.com/github/Goodman2442/test11/blob/main/%D0%A2%D1%80%D0%B0%D0%BD%D1%81%D0%BA%D1%80%D0%B8%D0%B1%D0%B0%D1%86%D0%B8%D1%8F_%7C_%D0%A0%D0%B0%D0%B7%D0%B1%D0%BE%D1%80_%D0%94%D0%97_%7C_%D0%A3%D0%98%D0%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Разбор домашнего задания по задаче исправление автоматической транскрибации.**


### Подключение Google Диска к Google Colab

Монтирование Google Диска к виртуальной машине Google Colab, предоставляя доступ к файлам на диске.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Установка зависимостей для использования PyTorch и Unsloth

Этот код устанавливает необходимые пакеты для работы с PyTorch и библиотекой Unsloth в зависимости от версии GPU, доступной в среде Google Colab.

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

### Загрузка и настройка метода FastLanguageModel для модели llama-3-8b-Instruct

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096  # Выберите любое значение! unsloth автоматически поддерживаем масштабирование RoPE!
dtype = None  # None для автоматического определения. Float16 для Tesla T4, V100, Bfloat16 для Ampere+.
load_in_4bit = True  # Используйте 4-битную квантизацию для уменьшения использования памяти. Можно установить False.

# Предварительно квантизированные модели в 4-битном формате, которые unsloth поддерживает для 4x более быстрой загрузки и отсутствия ошибок переполнения памяти (OOM).
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Инструкционная версия Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Инструкционная версия Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit",  # [НОВОЕ] 15 триллионов токенов Llama-3
    "unsloth/llama-3-70b-Instruct-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit" # Инструкционная версия Llama-3
]  # Больше моделей на https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # используйте токен, если используете защищенные модели, такие как meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Настройка модели FastLanguageModel с использованием PEFT

Добавленный адаптер LoRA, позволяет обновить только от 1 до 10% всех параметров!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Выберите любое число > 0! Рекомендуемые значения: 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,  # Поддерживается любое значение, но = 0 оптимально
    bias="none",     # Поддерживается любое значение, но = "none" оптимально
    # [НОВОЕ] "unsloth" использует на 30% меньше видеопамяти, позволяет использовать в 2 раза большие размеры батчей!
    use_gradient_checkpointing="unsloth",  # True или "unsloth" для очень длинных контекстов
    random_state=3407,
    use_rslora=False,  # Мы поддерживаем Rank Stabilized LoRA
    loftq_config=None,  # И LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Подготовка данных

### Подготовка данных для задачи исправления транскрибации

Этот код выполняет следующие действия:

1. Загружает данные из файлов `clean.txt` и `corrupt.txt`, содержащих корректные и искаженные строки соответственно.
2. Проверяет, что количество строк в обоих файлах одинаковое. Если это не так, поднимает ошибку.
3. Формирует список пар, где каждая пара состоит из искаженной строки и соответствующей ей корректной строки.
4. Разделяет данные на обучающую выборку (первые 1000 пар) и валидационную выборку (следующие 100 пар).
5. Определяет функцию `create_dataframe` для создания DataFrame с заданной инструкцией, входными и выходными данными.
6. Определяет инструкцию для задачи исправления транскрибации.
7. Создает DataFrame для обучающей и валидационной выборок, используя определенную функцию и инструкцию.
8. Сохраняет созданные DataFrame в формате CSV с именами `transcription_correction_train.csv` и `transcription_correction_validation.csv`.

Этот код обеспечивает подготовку данных для задачи исправления транскрибации.

In [ ]:
import pandas as pd
import random

# Загрузка данных из файлов
with open('/content/drive/MyDrive/Проекты/Train_dataset/clean.txt', 'r', encoding='utf8') as file:
    clean_lines = file.readlines()

with open('/content/drive/MyDrive/Проекты/Train_dataset/corrupt.txt', 'r', encoding='utf8') as file:
    corrupt_lines = file.readlines()

# Проверка, что количество строк в обоих файлах одинаковое
if len(clean_lines) != len(corrupt_lines):
    raise ValueError("Количество строк в файлах clean.txt и corrupt.txt должно быть одинаковым.")

# Формирование списка пар
pairs = list(zip(corrupt_lines, clean_lines))
# random.shuffle(pairs)

# Разделение на обучающую и валидационную выборки
train_pairs = pairs[:1000]
validation_pairs = pairs[1000:1100]

# Функция для создания DataFrame
def create_dataframe(pairs, instruction):
    return pd.DataFrame({
        'instruction': [instruction] * len(pairs),
        'input': [pair[0].strip() for pair in pairs],
        'output': [pair[1].strip() for pair in pairs],
    })

# Инструкция для задачи исправления транскрибации
instruction_text = 'Исправьте автоматическую транскрибацию (орфографию, термины, пунктуацию).'

# Создание DataFrame для обучающей выборки
train_df = create_dataframe(train_pairs, instruction_text)

# Создание DataFrame для валидационной выборки
validation_df = create_dataframe(validation_pairs, instruction_text)

# Сохранение DataFrame в формате CSV
train_df.to_csv('transcription_correction_train.csv', index=False, encoding='utf-8')
validation_df.to_csv('transcription_correction_validation.csv', index=False, encoding='utf-8')



Результат выполнения этого кода покажет первые несколько строк DataFrame, содержащего данные для задачи исправления транскрибации, включая инструкции, входные и выходные данные.

In [ ]:
import pandas as pd

df = pd.read_csv('transcription_correction_train.csv')
df

,instruction,input,output
0,Исправьте автоматическую транскрибацию (орфогр...,"Какс счита ьет эксперт, рояссиянам не стоит п...","Как считает эксперт, россиянам не стоит пережи..."
1,Исправьте автоматическую транскрибацию (орфогр...,"«Когдой идёт дождь илоеы снег, то усй водителя...","«Когда идёт дождь или снег, то у водителя нет ..."
2,Исправьте автоматическую транскрибацию (орфогр...,оВ теоориа в идеальное ситу ациая это должно у...,В теории в идеальной ситуации это должно учиты...
3,Исправьте автоматическую транскрибацию (орфогр...,Ранкее прленум Верхиовного судйые Россиая выпу...,Ранее пленум Верховного суда России выпустил п...
4,Исправьте автоматическую транскрибацию (орфогр...,тРуководитель Федерацоиаа автовладуельцев Росн...,Руководитель Федерации автовладельцев России С...
...,...,...,...
995,Исправьте автоматическую транскрибацию (орфогр...,От заболевани.й илые естественхныхт перичин.,От заболеваний или естественных причин.
996,Исправьте автоматическую транскрибацию (орфогр...,Одни из основных нашихк зардарчм — изученние п...,Одна из основных наших задач — изучение проявл...
997,Исправьте автоматическую транскрибацию (орфогр...,Но 90% нашей работы — из учеониеу фрагментогв ...,Но 90% нашей работы — изучение фрагментов ткан...
998,Исправьте автоматическую транскрибацию (орфогр...,аЗачастеую это влияет на ход лечеения.,Зачастую это влияет на ход лечения.


### Подготовка датасета для тонкой настройки модели

Этот код выполняет следующие действия:

1. Определяет системную инструкцию для редактора текста, который исправляет автоматическую транскрибацию.
2. Определяет шаблон `llama3_prompt` для форматирования данных.
3. Загружает данные из CSV файла в DataFrame с помощью `pandas`.
4. Конвертирует DataFrame в формат `Dataset` библиотеки Hugging Face.
5. Определяет функцию `formatting_prompts_func` для форматирования данных согласно шаблону.
6. Применяет функцию форматирования к датасету, создавая текстовые примеры.

Результат:
- Подготовленный датасет готов для использования в тонкой настройке модели.

Этот код обеспечивает правильную подготовку данных для последующего обучения модели.

In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd

system_instruction = """Вы - опытный редактор текста. Ваша задача - исправить автоматическую транскрибацию текста, чтобы он соответствовал правильной орфографии, использованию терминов и пунктуации. Вам будет предоставлен текст, полученный с помощью автоматической транскрибации (корректируемый текст), и ваш долг - отредактировать его так, чтобы он соответствовал правильному транскрибированному тексту (исправленный текст). Обратите внимание на следующие аспекты:

1. Орфография: Исправьте любые ошибки в написании слов.
2. Термины: Убедитесь, что все термины и специфические названия правильно транскрибированы.
3. Пунктуация: Восстановите правильное использование пунктуации, включая запятые, точки, кавычки и прочие знаки.

Отвечайте на русском языке!
"""

llama3_prompt = """<|start_header_id|>system<|end_header_id|>

{system_prompt}\n\n{instruction}<|eot_id|><|start_header_id|>user<|end_header_id|>

{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{output}<|eot_id|>"""

# Загрузите ваш датасет, если он уже в формате Hugging Face Datasets
# Если у вас данные в CSV, сначала загрузите их в pandas DataFrame, а затем конвертируйте в Dataset
data = pd.read_csv("/content/transcription_correction_train.csv")
formatted_dataset = Dataset.from_pandas(data)


# Форматирование датасета
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = llama3_prompt.format(system_prompt=system_instruction, instruction=instruction, input=input, output=output)
        texts.append(text)
    return { "text": texts, }
pass

# Применяем форматирование
dataset = formatted_dataset.map(formatting_prompts_func, batched=True)

# Теперь dataset готов к использованию для fine-tuning модели.

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Этот код демонстрирует токенизацию первого текстового примера из форматированного датасета, используя токенизатор.

1. Извлекает первый текстовый пример из форматированного датасета.
2. Применяет токенизатор для преобразования текста в токены.
3. Возвращает токенизированный результат.

Этот шаг показывает как превращается текст в последовательность токенов, понятных модели.

In [ ]:
tokenizer(dataset[0]['text'])

{'input_ids': [128000, 128006, 9125, 128007, 271, 46710, 482, 127576, 34613, 117105, 37773, 71995, 1506, 13, 23784, 123572, 53280, 50223, 1506, 482, 53687, 28086, 18264, 105332, 8131, 81301, 105779, 104875, 66144, 31203, 10124, 30436, 12182, 71995, 1506, 11, 82250, 99845, 106445, 111442, 106668, 39988, 109773, 19916, 103752, 47273, 11, 89327, 91459, 102031, 115275, 6856, 7740, 118810, 89348, 54686, 13, 23784, 24697, 69923, 115739, 61498, 71995, 11, 52432, 106253, 5524, 109344, 105332, 8131, 110740, 104875, 66144, 31203, 10124, 54686, 320, 102161, 113023, 23311, 20346, 10298, 35723, 71995, 705, 7740, 98117, 101140, 15752, 482, 20879, 44075, 16248, 67589, 18482, 73385, 50945, 11, 82250, 99845, 106445, 111442, 106668, 101322, 104875, 66144, 31203, 10124, 59921, 7486, 101322, 71995, 3865, 320, 13810, 44646, 127542, 71995, 570, 80798, 81659, 27055, 113842, 13373, 120306, 120391, 30656, 4655, 1473, 16, 13, 115651, 19916, 114173, 25, 120886, 28086, 109121, 102376, 51736, 89763, 17165, 5927, 1

#### Подготовка валидационного датасета для тонкой настройки модели

Этот код выполняет следующие действия:

1. Загружает данные валидационного датасета из CSV файла в DataFrame с помощью `pandas`.
2. Конвертирует DataFrame в формат `Dataset` библиотеки Hugging Face.
3. Применяет функцию форматирования к валидационному датасету.

Результат:
- Подготовленный валидационный датасет готов для использования в процессе тонкой настройки модели.

Этот код позволяет подготовить валидационный датасет для последующего использования в процессе оценки точности модели.

In [ ]:
# Загрузите ваш датасет, если он уже в формате Hugging Face Datasets
# Если у вас данные в CSV, сначала загрузите их в pandas DataFrame, а затем конвертируйте в Dataset
data_eval = pd.read_csv("/content/transcription_correction_validation.csv")
formatted_dataset = Dataset.from_pandas(data_eval)

# Применяем форматирование
eval_dataset = formatted_dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]


## Обучение модели

**Текущие настройки**

**per_device_train_batch_size**: размер батча для одного устройства (**16**).
**gradient_accumulation_steps**: количество шагов для накопления градиентов (**4**)
**были выбраны для видеокарты A100**.

**Для T4 рекомендуем:**

**per_device_train_batch_size = 2**

**gradient_accumulation_steps = 4**

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=10,
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
#@title Показать текущую статистику памяти
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
5.605 GB of memory reserved.


Запуск обучения

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 4
\        /    Total batch size = 64 | Total steps = 150
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.213900
2,2.197800
3,2.167300
4,2.083500
5,1.870300
6,1.602600
7,1.353300
8,1.103200
9,0.954000
10,0.955600


In [ ]:
#@title Показать итоговую статистику памяти и времени
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1242.7555 seconds used for training.
20.71 minutes used for training.
Peak reserved memory = 16.248 GB.
Peak reserved memory for training = 10.643 GB.
Peak reserved memory % of max memory = 41.068 %.
Peak reserved memory for training % of max memory = 26.901 %.


## Предсказание (инференс)

Загрузка валидационной выборки

In [ ]:
import pandas as pd

df = pd.read_csv('/content/transcription_correction_validation.csv')
df

,instruction,input,output
0,Исправьте автоматическую транскрибацию (орфогр...,«аЗа рядье» — зн аковыеи символ вМкосквы с уни...,«Зарядье» — знаковый символ Москвы с уникальны...
1,Исправьте автоматическую транскрибацию (орфогр...,"По его словйам, за вйремяя руаботы парк тзавое...","По его словам, за время работы парк завоевал п..."
2,Исправьте автоматическую транскрибацию (орфогр...,"Трдехлеб также наппомнеил, чтео «Зарят,дье» — ...","Трехлеб также напомнил, что «Зарядье» — первый..."
3,Исправьте автоматическую транскрибацию (орфогр...,Ранее телеканал з«3606» сообщал о фовтоввыстав...,Ранее телеканал «360» сообщал о фотовыставке «...
4,Исправьте автоматическую транскрибацию (орфогр...,"Калкк сообщаетшб официальные сайт клуба, Щерба...","Как сообщает официальный сайт клуба, Щербак по..."
...,...,...,...
95,Исправьте автоматическую транскрибацию (орфогр...,"я«Частичноп признааю вину в хищении»», — сказа...","«Частично признаю вину в хищении», — сказал он..."
96,Исправьте автоматическую транскрибацию (орфогр...,"Мужчинами дообавил, чвто говфокрит только про ...","Мужчина добавил, что говорит только про дело «..."
97,Исправьте автоматическую транскрибацию (орфогр...,Ранее в Москвееи задержала 26-летнего мужчину...,"Ранее в Москве задержали 26-летнего мужчину, п..."
98,Исправьте автоматическую транскрибацию (орфогр...,"В егоВ квартире обнаруожилые феназепам, клозап...","В его квартире обнаружили феназепам, клозапин ..."


Генерация ответа 3 строки валидационного датафрейма

In [ ]:
idx_row = 3

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    llama3_prompt.format(
        system_prompt=system_instruction, #
        instruction=df.loc[idx_row]['instruction'], # instruction
        input=df.loc[idx_row]['input'], # input
        output="", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True, temperature=0.7)
completion = tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print('Исходный текст: ', df.loc[idx_row]['input'])
print('Результат: ', completion[0].split('<|start_header_id|>assistant<|end_header_id|>\n\n')[-1].replace('<|eot_id|>', ''))

Исходный текст:  Ранее телеканал з«3606» сообщал о фовтоввыставке «Звёзды  МАКеС в парке «Зарядье.».
Результат:  Ранее телеканал «360» сообщал о выставке «Звёзды МАК в парке «Зарядье».


Генерация ответа 99 строки валидационного датафрейма

In [ ]:
idx_row = 99

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    llama3_prompt.format(
        system_prompt=system_instruction, #
        instruction=df.loc[idx_row]['instruction'], # instruction
        input=df.loc[idx_row]['input'], # input
        output="", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True, temperature=0.7)
completion = tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print('Исходный текст: ', df.loc[idx_row]['input'])
print('Результат: ', completion[0].split('<|start_header_id|>assistant<|end_header_id|>\n\n')[-1].replace('<|eot_id|>', ''))

Исходный текст:  МВД переквалифицировало деййстовия с«московского отравителяс со статьые « Кража» лна «Разбой».
Результат:  МВД переквалифицировало действия «московского отравителя со статьей «Кража» на «Разбой».


Вы также можете использовать TextStreamer для непрерывного вывода - чтобы вы могли видеть генерацию токена за токеном, а не ждать все время!

In [ ]:
from transformers import TextStreamer

idx_row = 77

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
        llama3_prompt.format(
        system_prompt=system_instruction, #
        instruction=df.loc[idx_row]['instruction'], # instruction
        input=df.loc[idx_row]['input'], # input
        output="", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024, temperature=0.0)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Вы - опытный редактор текста. Ваша задача - исправить автоматическую транскрибацию текста, чтобы он соответствовал правильной орфографии, использованию терминов и пунктуации. Вам будет предоставлен текст, полученный с помощью автоматической транскрибации (корректируемый текст), и ваш долг - отредактировать его так, чтобы он соответствовал правильному транскрибированному тексту (исправленный текст). Обратите внимание на следующие аспекты:

1. Орфография: Исправьте любые ошибки в написании слов.
2. Термины: Убедитесь, что все термины и специфические названия правильно транскрибированы.
3. Пунктуация: Восстановите правильное использование пунктуации, включая запятые, точки, кавычки и прочие знаки.

Отвечайте на русском языке!


Исправьте автоматическую транскрибацию (орфографию, термины, пунктуацию).<|eot_id|><|start_header_id|>user<|end_header_id|>

Москвса ои область, что это за сэубътект? Это оодин субъект.<|eot_id|><|start_h

Генерация ответа 33 строки валидационного датафрейма

In [ ]:
idx_row = 33

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
        llama3_prompt.format(
        system_prompt=system_instruction, #
        instruction=df.loc[idx_row]['instruction'], # instruction
        input=df.loc[idx_row]['input'], # input
        output="", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1024, temperature=0.7)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Вы - опытный редактор текста. Ваша задача - исправить автоматическую транскрибацию текста, чтобы он соответствовал правильной орфографии, использованию терминов и пунктуации. Вам будет предоставлен текст, полученный с помощью автоматической транскрибации (корректируемый текст), и ваш долг - отредактировать его так, чтобы он соответствовал правильному транскрибированному тексту (исправленный текст). Обратите внимание на следующие аспекты:

1. Орфография: Исправьте любые ошибки в написании слов.
2. Термины: Убедитесь, что все термины и специфические названия правильно транскрибированы.
3. Пунктуация: Восстановите правильное использование пунктуации, включая запятые, точки, кавычки и прочие знаки.

Отвечайте на русском языке!


Исправьте автоматическую транскрибацию (орфографию, термины, пунктуацию).<|eot_id|><|start_header_id|>user<|end_header_id|>

уКроме того, российский алидер проведёйт поереговоры с презирденйтом Франции Эм

Тестирование на подборке примеров из валидационной выборки с сохранением результатов в папку проекта на гугл диске /content/drive/MyDrive/Проекты/Train_dataset/

In [ ]:
import pandas as pd

model_name = 'llama-3-8b-Instruct-bnb-4bit'

temperature = 0.0

file_path = "/content/transcription_correction_validation.csv" #@param {type:"string"}
file_name = file_path.split('/')[-1].split('.')[0]
df = pd.read_csv(file_path)

new_data = []

for index, row in df.iterrows():
  input = row['input']
  output = row['output']
  instruction = row['instruction']
  system_instruction = """Вы - опытный редактор текста. Ваша задача - исправить автоматическую транскрибацию текста, чтобы он соответствовал правильной орфографии, использованию терминов и пунктуации. Вам будет предоставлен текст, полученный с помощью автоматической транскрибации (корректируемый текст), и ваш долг - отредактировать его так, чтобы он соответствовал правильному транскрибированному тексту (исправленный текст). Обратите внимание на следующие аспекты:

  1. Орфография: Исправьте любые ошибки в написании слов.
  2. Термины: Убедитесь, что все термины и специфические названия правильно транскрибированы.
  3. Пунктуация: Восстановите правильное использование пунктуации, включая запятые, точки, кавычки и прочие знаки.

  Отвечайте на русском языке!"""

  FastLanguageModel.for_inference(model) # Enable native 2x faster inference
  inputs = tokenizer(
  [
          llama3_prompt.format(
          system_prompt=system_instruction, #
          instruction=row['instruction'], # instruction
          input=row['input'], # input
          output="", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True, temperature=temperature)
  completion = tokenizer.batch_decode(outputs)

  new_data.append({'input': row['input'],
                   "output_base": row['output'],
                   'result': completion[0].split('<|start_header_id|>assistant<|end_header_id|>\n\n')[-1].replace('<|eot_id|>', ''),
                   "Оценка": None,
                   "Комментарий": None})
  print(completion[0].split('<|start_header_id|>assistant<|end_header_id|>\n\n')[-1].replace('<|eot_id|>', ''))

new_df = pd.DataFrame(new_data)
new_df.to_csv(f'/content/drive/MyDrive/Проекты/Train_dataset/result_{model_name}_{file_name}.csv', index=False)
new_df.to_excel(f'/content/drive/MyDrive/Проекты/Train_dataset/result_{model_name}_{file_name}.xlsx', index=False)
print("Завершено!")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


«За ряд» — знаковый символ Москвы с уникальным ландшафтом и необычной архитектурой, который является научно-просветительским и культурным проектом», — сказал он.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


По его словам, за время работы парк завоевал престижные международные премии.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Мужчинами добавил, что критика только про сделали «ве отношения трёх» человек, семей которых не знает.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Ранее в Москве задержали 26-летнего мужчину, подозреваемого во взяточничестве людей напитки и их последующем ограблении.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


В ее квартире обнаружили феназиапин, клонапин и атропин.
МВД переквалифицировало действия «московского отравителя» на статьи «Кража» и «Разбой».
Завершено!


In [ ]:
new_df

,input,output_base,result,Оценка,Комментарий
0,«аЗа рядье» — зн аковыеи символ вМкосквы с уни...,«Зарядье» — знаковый символ Москвы с уникальны...,«За ряд» — знаковый символ Москвы с уникальным...,None,None
1,"По его словйам, за вйремяя руаботы парк тзавое...","По его словам, за время работы парк завоевал п...","По его словам, за время работы парк завоевал п...",None,None
2,"Трдехлеб также наппомнеил, чтео «Зарят,дье» — ...","Трехлеб также напомнил, что «Зарядье» — первый...","Трёхбок также напомнил, что «Зарядье» — первая...",None,None
3,Ранее телеканал з«3606» сообщал о фовтоввыстав...,Ранее телеканал «360» сообщал о фотовыставке «...,Ранее телеканал «360» сообщал о фотовыставке «...,None,None
4,"Калкк сообщаетшб официальные сайт клуба, Щерба...","Как сообщает официальный сайт клуба, Щербак по...","Кроме того, Шербаак подписал соглашение с «Кяt...",None,None
...,...,...,...,...,...
95,"я«Частичноп признааю вину в хищении»», — сказа...","«Частично признаю вину в хищении», — сказал он...","«Частично признаю вину в хищении», — сказал он...",None,None
96,"Мужчинами дообавил, чвто говфокрит только про ...","Мужчина добавил, что говорит только про дело «...","Мужчинами добавил, что критика только про сдел...",None,None
97,Ранее в Москвееи задержала 26-летнего мужчину...,"Ранее в Москве задержали 26-летнего мужчину, п...","Ранее в Москве задержали 26-летнего мужчину, п...",None,None
98,"В егоВ квартире обнаруожилые феназепам, клозап...","В его квартире обнаружили феназепам, клозапин ...","В ее квартире обнаружили феназиапин, клонапин ...",None,None


## Сохранение, загрузка настроенных моделей

Чтобы сохранить окончательную модель в виде адаптеров LoRA, используйте либо push_to_hub от Huggingface для онлайн-сохранения, либо save_pretrained для локального сохранения.

**[ПРИМЕЧАНИЕ]** Это сохраняет ТОЛЬКО адаптеры LoRA, а не полную модель. Чтобы сохранить в 16-битном формате или GGUF, покажем далее!

### Сохранение настроенной модели

Этот код выполняет следующие действия:

1. Сохраняет настроенную модель локально в директорию "lora_model".
2. (Закомментировано) Опционально, может загрузить модель на онлайн хаб, если раскомментировать строку и указать имя модели и токен.

Локальное сохранение позволяет сохранить изменения и использовать модель на локальной машине, в то время как онлайн сохранение делает модель доступной для других через Huggingface hub.

In [ ]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
!zip -r lora_model_llama3_instr.zip '/content/lora_model'

  adding: content/lora_model/ (stored 0%)
  adding: content/lora_model/README.md (deflated 66%)
  adding: content/lora_model/adapter_config.json (deflated 53%)
  adding: content/lora_model/adapter_model.safetensors (deflated 8%)


**Архивирование настроенной модели**

Этот код выполняет следующие действия:

- Создает zip-архив `lora_model_llama3_instr.zip` из директории `/content/lora_model`.

Архивирование позволяет легко передавать и хранить настроенную модель.

In [ ]:
!cp /content/lora_model_llama3_instr.zip /content/drive/MyDrive/Проекты/Train_llms/lora_model_llama3_8b_instr_transcribe_23_05_2024.zip